In [1]:
import torch
from torch.utils.data import DataLoader
from generate_data import generate_test_data, decoder
import deqmodel
import basemodel

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
data = generate_test_data()
mydataloader = DataLoader(data, batch_size=len(data))
for feats, labels in mydataloader:
    X = feats.to(device).float()
    y = labels.to(device).float()

In [6]:
base_state = torch.load('basemodel.pth')
base = basemodel.NeuralNetwork().to(basemodel.device)
base.load_state_dict(base_state)
base.eval()
with torch.no_grad():
    pred, hidden = base(X)

In [8]:
bool_arr = pred > 0
prediction = torch.Tensor([1 if x else 0 for x in bool_arr]).to(device)


In [9]:
def stats(X, y, pred):
    tp, tn, fp, fn = 0, 0, 0, 0
    fp_expr, fn_expr = [], []
    for i in range(len(y)):
        if pred[i] == 1:
            if pred[i] == y[i]:
                tp += 1
            else:
                fp += 1
                fp_expr.append(decoder(X[i]))
        else:
            if pred[i] == y[i]:
                tn += 1
            else:
                fn += 1
                fn_expr.append(decoder(X[i]))
    return tp, tn, fp, fn, fp_expr, fn_expr

In [17]:
tp, tn, fp, fn, fpx, fnx = stats(X, y, prediction)
print(fp, fn)

184 918


In [18]:
deq_state = torch.load('deqmodel.pth')
deq = deqmodel.NeuralNetwork().to(deqmodel.device)
deq.load_state_dict(deq_state)
deq.eval()
with torch.no_grad():
    pred = deq(X)

/home/ryanl/gcloud/deqs/deqmodel.py:66: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:859.)
  alpha = torch.solve(y[:, :n + 1], H[:, :n + 1, :n + 1])[0][:, 1:n + 1, 0]  # (bsz x n)


In [20]:
bool_arr = pred > 0
prediction = torch.Tensor([1 if x else 0 for x in bool_arr]).to(device)
tp, tn, fp, fn, fpx, fnx = stats(X, y, prediction)
print(fp, fn)

182 412
